In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install scikit-maad
!pip install pydub

In [ ]:
import os
from pathlib import Path
from pydub import AudioSegment
import pandas as pd
from maad import sound, util, rois, features
from maad.util import (power2dB, plot2d, format_features, read_audacity_annot,
                       overlay_rois, overlay_centroid, crop_image,save_figlist)

In [ ]:
def grab_audio(path, audio_format='mp3'):
  audio_format_new = 'wav'
  filelist = []
  for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            if name[-3:].casefold() == audio_format_new and name[:2] != '._':
                filelist.append(os.path.join(root, name))
            elif name[-3:].casefold() == audio_format and name[:2] != '._':
              try:
                sound = AudioSegment.from_mp3(os.path.join(root, name))
                sound = sound.set_channels(1)
                sound.export(os.path.join(root, name.replace(".mp3",".wav").replace(".MP3",".wav")), format="wav")
                os.remove(os.path.join(root, name))
                filelist.append(os.path.join(root, name.replace(".mp3",".wav").replace(".MP3",".wav")))
              except:
                os.remove(os.path.join(root, name))
              
  return filelist

In [ ]:
AUDIOS_ESPECIES = 300
SOBREESCRIBIR = True
#Creamos (si no existe ya) la carpeta en la que vamos a almacenaar los archivos
path = os.path.join('drive',"MyDrive","Colab Notebooks", "Cuaderno1","DataSet")
original=os.path.join(path,"Original")
entrenamiento=os.path.join(original,"train")
test=os.path.join(original,"test")
val=os.path.join(original,"val")
existe = os.path.exists(entrenamiento)#Se optiene si existe o no
if not existe:  
  os.makedirs(entrenamiento) # Crea un nuevo repositorio porque no existe
existe = os.path.exists(test)#Se optiene si existe o no
if not existe:  
  os.makedirs(test) # Crea un nuevo repositorio porque no 
XC_ROOTDIR = '/content/drive/MyDrive/Colab Notebooks/Cuaderno1/DataSet/'
XC_DIR = 'Original/train/'


In [2]:
data = [
        ['Escribano soteño',    'Emberiza cirlus'],
        ['Verdecillo',     	    'Serinus serinus'],
        ['Escribano cerillo',   'Emberiza citrinella'],
       ['Verderón europeo',    'Chloris chloris'],
       ['Jilguero europeo',    'Carduelis carduelis'],
        ['Pinzón vulgar',       'Fringilla coelebs'],
        ['Oropéndola europea​',  'Oriolus oriolus'],
        ['Agateador común',     'Certhia brachydactyla'],
        ['Agateador norteño',   'Certhia familiaris'],
        ['Herrerillo capuchino','Lophophanes cristatus'],
        ['Carbonero palustre',  'Poecile palustris'],
        ['Herrerillo común',    'Cyanistes caeruleus'],
         ['Carbonero garrapinos','Periparus ater'],
         ['Carbonero común',     'Parus major'],
         ['Reyezuelo listado',   'Regulus ignicapilla'],
         ['Reyezuelo sencillo',  'Regulus regulus'],
         ['Mosquitero ibérico',  'Phylloscopus ibericus'],
         ['Mosquitero papialbo', 'Phylloscopus bonelli'],
         ['mosuitero musical',  'Phylloscopus trochilus'],
         ['colirrojo tizón',     'Phoenicurus ochruros'],
        ]

In [ ]:
df_species = pd.DataFrame(data,columns =['english name',
                                         'scientific name'])
gen = []
sp = []
for name in df_species['scientific name']:
    gen.append(name.rpartition(' ')[0])
    sp.append(name.rpartition(' ')[2])

In [ ]:
df_query = pd.DataFrame()
df_query['param1'] = gen
df_query['param2'] = sp
df_query['param3'] ='type:song'
df_query['param4'] ='area:europe'
#df_query['param5'] ='cnt:spain'
# df_query['param6'] ='len_gt:5'
# df_query['param7'] ='q_gt:C'

# Get recordings metadata corresponding to the query
df_datasetC= util.xc_multi_query(df_query, 
                                 format_time = False,
                                 format_date = False,
                                 verbose=True)

Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Emberiza%20cirlus%20type:song%20area:europe&page=1
Loading page 2...
https://www.xeno-canto.org/api/2/recordings?query=Emberiza%20cirlus%20type:song%20area:europe&page=2
Found 2 pages in total.
Saved metadata for 524 files
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Serinus%20serinus%20type:song%20area:europe&page=1
Loading page 2...
https://www.xeno-canto.org/api/2/recordings?query=Serinus%20serinus%20type:song%20area:europe&page=2
Found 2 pages in total.
Saved metadata for 770 files
Loading page 1...
https://www.xeno-canto.org/api/2/recordings?query=Emberiza%20citrinella%20type:song%20area:europe&page=1
Loading page 2...
https://www.xeno-canto.org/api/2/recordings?query=Emberiza%20citrinella%20type:song%20area:europe&page=2
Loading page 3...
https://www.xeno-canto.org/api/2/recordings?query=Emberiza%20citrinella%20type:song%20area:europe&page=3
Loading page 4...
https://www.xeno-canto.org/api/

In [ ]:
df_dataset = util.xc_selection(df_datasetC,
                               max_nb_files=AUDIOS_ESPECIES,
                               max_length='02:00',
                               min_length='00:10',
                               min_quality='B',
                               verbose = True )

Emberiza cirlus
    ... request 300 files of quality A
    --> found 205 files of quality A and 00:10<length<02:00
    ... request 95 files of quality B
    --> found 95 files of quality B and 00:10<length<02:00
    total files : 300
-----------------------------------------
Serinus serinus
    ... request 300 files of quality A
    --> found 211 files of quality A and 00:10<length<02:00
    ... request 89 files of quality B
    --> found 89 files of quality B and 00:10<length<02:00
    total files : 300
-----------------------------------------
Emberiza citrinella
    ... request 300 files of quality A
    --> found 288 files of quality A and 00:10<length<02:00
    ... request 12 files of quality B
    --> found 12 files of quality B and 00:10<length<02:00
    total files : 300
-----------------------------------------
Chloris chloris
    ... request 300 files of quality A
    --> found 300 files of quality A and 00:10<length<02:00
    total files : 300
-------------------------------

In [ ]:
util.xc_download(df_dataset,
                 rootdir = XC_ROOTDIR,
                 dataset_name= XC_DIR,
                 overwrite=SOBREESCRIBIR,
                 save_csv= False,
                 verbose = True)

Se han truncado las últimas 5000 líneas del flujo de salida.
Saving file  842 / 5824 : https://xeno-canto.org/741893/download
Saving file  843 / 5824 : https://xeno-canto.org/402005/download
Saving file  844 / 5824 : https://xeno-canto.org/102106/download
Saving file  845 / 5824 : https://xeno-canto.org/240221/download
Saving file  846 / 5824 : https://xeno-canto.org/627541/download
Saving file  847 / 5824 : https://xeno-canto.org/722883/download
Saving file  848 / 5824 : https://xeno-canto.org/357754/download
Saving file  849 / 5824 : https://xeno-canto.org/700132/download
Saving file  850 / 5824 : https://xeno-canto.org/31637/download
Saving file  851 / 5824 : https://xeno-canto.org/411287/download
Saving file  852 / 5824 : https://xeno-canto.org/174223/download
Saving file  853 / 5824 : https://xeno-canto.org/644462/download
Saving file  854 / 5824 : https://xeno-canto.org/635611/download
Saving file  855 / 5824 : https://xeno-canto.org/635039/download
Saving file  856 / 5824 : http

In [ ]:
filelist = grab_audio(entrenamiento)

In [ ]:
df = pd.DataFrame()
for file in filelist:
    df = df.append({'fullfilename': file,
                    'filename': Path(file).parts[-1][:-4],
                    'species': Path(file).parts[-2]},
                   ignore_index=True)

print('=====================================================')
print('number of files : %2.0f' % len(df))
print('number of species : %2.0f' % len(df.species.unique()))
print('=====================================================')

In [ ]:
import random
def todoAEntrenamiento():
  for nombre_directorio, dirs, ficheros in os.walk(test.replace("Original","Espectrogramas")):
    for nombre_fichero in ficheros:
      print(nombre_fichero," a train.")
      os.replace(os.path.join(nombre_directorio,nombre_fichero),os.path.join(nombre_directorio.replace("test","train"),nombre_fichero))
  for nombre_directorio, dirs, ficheros in os.walk(val.replace("Original","Espectrogramas")):
    for nombre_fichero in ficheros:
      print(nombre_fichero," a train.")
      os.replace(os.path.join(nombre_directorio,nombre_fichero),os.path.join(nombre_directorio.replace("val","train"),nombre_fichero))
def ordena(e, t):
  todoAEntrenamiento()
  for nombre_directorio, dirs, ficheros in os.walk(e):
    idx = 0
    test = nombre_directorio.replace("train","test")
    val = nombre_directorio.replace("train","val")
    existe = os.path.exists(test)#Se optiene si existe o no
    if not existe:  
        os.makedirs(test)
    existe = os.path.exists(val)#Se optiene si existe o no
    if not existe:  
        os.makedirs(val)
    random.shuffle(ficheros)
    for nombre_fichero in ficheros:
      if idx > len(ficheros) * .9 :
        print(nombre_fichero," a Test.")
        os.replace(os.path.join(nombre_directorio,nombre_fichero),os.path.join(nombre_directorio.replace("train","test"),nombre_fichero))
      elif(idx > len(ficheros) * .8):
        print(nombre_fichero," a Val.")
        os.replace(os.path.join(nombre_directorio,nombre_fichero),os.path.join(nombre_directorio.replace("train","val"),nombre_fichero))
      idx = idx +1
ordena(entrenamiento.replace("Original","Espectrogramas"),test.replace("Original","Espectrogramas"))

37.095298911857064.png  a Val.
35.86690508056156.png  a Val.
30.589124349352428.png  a Val.
30.860399047558204.png  a Val.
37.102071141489816.png  a Val.
39.77645181603873.png  a Val.
31.372323057241328.png  a Val.
42.471676183545796.png  a Val.
36.48510476993425.png  a Val.
38.70847964769699.png  a Val.
30.44415133685108.png  a Val.
32.29181892461073.png  a Val.
40.5820720813952.png  a Val.
36.009069733602914.png  a Val.
32.5418960322964.png  a Val.
38.64958677461818.png  a Val.
36.63998339772219.png  a Val.
30.01293695414858.png  a Val.
32.336297390990445.png  a Val.
38.782291010258234.png  a Val.
39.702888523034666.png  a Val.
41.585295031738525.png  a Val.
38.01677078913738.png  a Val.
33.6639395640395.png  a Val.
31.701260830300907.png  a Val.
33.38193732623128.png  a Val.
36.77212574041549.png  a Val.
35.68472014022443.png  a Test.
32.63814870152729.png  a Test.
33.105692908811164.png  a Test.
34.61711837556985.png  a Test.
36.26983175188653.png  a Test.
31.68373818373052.png  a 